In [7]:
import json
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate


def get_weather(longitude, latitude):
    print(f"weather for longitude({longitude}) and latitude({latitude})")


function = {
    "name": "get_weather",
    "description": """
    function that takes longitude and latitude to find the weather of a place
    """,
    "parameters": {
        "type": "object",
        "properties": {
            "longitude": {
                "type": "string",
                "description": "The longitude coordinate",
            },
            "latitude": {
                "type": "string",
                "description": "The latitude coordinate",
            },
        },
    },
    "required": ["longitude", "latitude"],
}

llm = ChatOpenAI(temperature=0.1).bind(
    function_call="auto",
    functions=[function],
)

prompt = PromptTemplate.from_template("Who is the weather in {city}")

chain = prompt | llm

response = chain.invoke({"city": "rome"})

response = response.additional_kwargs["function_call"]["arguments"]

response = json.loads(response)

get_weather(response["longitude"], response["latitude"])

weather for longitude(12.4964) and latitude(41.9028)


In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate


function = {
    "name": "create_quiz",
    "description": "function that takes a list of questions and answers and returns a quiz",
    "parameters": {
        "type": "object",
        "properties": {
            "questions": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                        },
                        "answers": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "answer": {
                                        "type": "string",
                                    },
                                    "correct": {
                                        "type": "boolean",
                                    },
                                },
                                "required": ["answer", "correct"],
                            },
                        },
                    },
                    "required": ["question", "answers"],
                },
            }
        },
        "required": ["questions"],
    },
}


llm = ChatOpenAI(
    temperature=0.1,
).bind(
    function_call={
        "name": "create_quiz",
    },
    functions=[
        function,
    ],
)

prompt = PromptTemplate.from_template("Make a quiz about {city}")

chain = prompt | llm

response = chain.invoke({"city": "rome"})


response = response.additional_kwargs["function_call"]["arguments"]

for question in json.loads(response)["questions"]:
    print(question)

{'question': 'What year was Rome founded?', 'answers': [{'answer': '753 BC', 'correct': True}, {'answer': '476 AD', 'correct': False}, {'answer': '1492 AD', 'correct': False}, {'answer': '1066 AD', 'correct': False}]}
{'question': 'Who was the first emperor of Rome?', 'answers': [{'answer': 'Julius Caesar', 'correct': False}, {'answer': 'Augustus', 'correct': True}, {'answer': 'Nero', 'correct': False}, {'answer': 'Constantine', 'correct': False}]}
{'question': 'What was the official language of the Roman Empire?', 'answers': [{'answer': 'Latin', 'correct': True}, {'answer': 'Greek', 'correct': False}, {'answer': 'Italian', 'correct': False}, {'answer': 'Spanish', 'correct': False}]}
{'question': 'Which famous Roman general led the conquest of Gaul?', 'answers': [{'answer': 'Hannibal', 'correct': False}, {'answer': 'Scipio Africanus', 'correct': False}, {'answer': 'Julius Caesar', 'correct': True}, {'answer': 'Mark Antony', 'correct': False}]}
{'question': 'What architectural marvel di